In [ ]:
# Let's get comfortable first
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [ ]:
import pandas as pd
import openpyxl
from pathlib import Path
import math
import os

In [ ]:
from useful_functions import create_pedigree_matrix 

In [ ]:
# Define paths as constants
LCI_DIR = 'data/LCI'
EI_DIR = Path("C:/Users/mp_ma/AppData/Local/pylca/EcoinventInterface/cache/ecoinvent 3.9.1_cutoff_ecoSpold02/datasets")

# Setting project and databases

In [ ]:
ei_name = "ecoinvent-3.9.1-cutoff"

In [ ]:
bd.projects
#sorted(bd.projects)

In [ ]:
bd.projects.set_current("regioinvent")
#bd.projects.delete_project(name='excel importer', delete_dir=True)
#bd.projects.rename("<new_project_name>")

In [ ]:
bd.databases

In [ ]:
# When we execute this cell, we will check if it's already been imported, and if not (else) we import it.

if ei_name in bd.databases:
    print("Database has already been imported.")
else:
# Go ahead and import:
    ei_importer = bi.SingleOutputEcospold2Importer(EI_DIR, ei_name)
    # Apply stragegies 
    ei_importer.apply_strategies()
    # We can get some statistics
    ei_importer.statistics()
    # Now we will write the database into our project. 
    ei_importer.write_database()

# LCI from Istrate et al (2024)

In [ ]:
# Import LIB raw materials LCIs
lci_lib_rms = bw.ExcelImporter(r'data/LCI/lci_LIB_raw_materials.xlsx')
lci_lib_rms.apply_strategies()
lci_lib_rms.match_database("ecoinvent-3.9.1-cutoff", fields=('name', 'reference product', 'unit', 'location'))
lci_lib_rms.match_database("biosphere3", fields=('name', 'unit', 'categories'))

In [ ]:
# Add uncertainty data for pedigree matrix
for ds in lci_lib_rms.data:
    for exc in ds["exchanges"]:
        if "pedigree" in exc:

            # Pedigree are stored as strings
            pedigree_str = exc["pedigree"].strip("()")
            pedigre_scores = tuple([int(x) for x in pedigree_str.split(", ")])
            exc_amount = exc["amount"]

            uncertainty_dict = create_pedigree_matrix(pedigre_scores, exc_amount)
            exc.update(uncertainty_dict)

In [ ]:
lci_lib_rms.statistics()

In [ ]:
[u for u in lci_lib_rms.unlinked if u["type"] == "biosphere"]

In [ ]:
[u for u in lci_lib_rms.unlinked if u["type"] == "technosphere"]

## Migration from EI 3.10 to 3.9.1

We do that to ensure it can be compatible with regioinvent and IW2.1+ regionalized

In [ ]:
migration_391 = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("market for soda ash, light", 
             "soda ash, light", 
             "ROW"),
            {"location": "GLO"}
        ),

        (
            ("sodium hydroxide to generic market for neutralising agent", 
             "neutralising agent, sodium hydroxide-equivalent", 
             "ROW"),
            {"location": "GLO"}
        ),
        
        (
            ("market for sodium hydroxide, without water, in 50% solution state", 
             "sodium hydroxide, without water, in 50% solution state", 
             "ROW"),
            {"location": "GLO"}
        ),
        
        (
            ("market for hydrogen, gaseous, medium pressure, merchant", 
             "hydrogen, gaseous, medium pressure, merchant", 
             "ROW"),
            {"name": "market for hydrogen, gaseous",
             "reference product": "hydrogen, gaseous",
                "location": "GLO"}
        ),

        (
            ("market for coke", 
             "coke", 
             "ROW"),
            {"location": "GLO"}
        ),
        
        ( # Don't know what is happening with that one
            ("market for iron(III) chloride, without water, in 40% solution state", 
             "iron(III) chloride, without water, in 40% solution state", 
             "GLO"),
            {"reference product": "iron (III) chloride, without water, in 40% solution state"}
        ),
        
        (
            ("market for nickel smelter slag", 
             "nickel smelter slag", 
             "ROW"),
            {"location": "GLO"}
        ),
        
        ( ## incorrect but it's to avoid having errors, seems like a new activity in EI3.10
            ("coke production, wet quenching", 
             "coal tar", 
             "CN"),
            {"name": "coking",
            "location": "ROW"}
        ),
        
        ( 
            ("market for coke", 
             "coke", 
             "CN"),
            {"location": "GLO"}
        ),
        
        (
            ("Manganese", "", "", ("water", "ocean",)),
            {
                "name": "Manganese II",
            },
        ), 

        (
            ("Manganese", "", "", ("water",)),
            {
                "name": "Manganese II",
            },
        ), 
    ],
}

In [ ]:
bi.Migration(name="ei3.10-3.9.1").write(data=migration_391, description="ei 3.10 to 3.9.1")
"ei3.10-3.9.1" in bi.migrations
bi.Migration("ei3.10-3.9.1")

In [ ]:
lci_lib_rms.data = bi.strategies.migrate_exchanges(
    db=lci_lib_rms.data,
    migration="ei3.10-3.9.1"
)

In [ ]:
lci_lib_rms.match_database("ecoinvent-3.9.1-cutoff", fields=('name', 'reference product', 'unit', 'location'))
lci_lib_rms.match_database("biosphere3", fields=('name', 'unit', 'categories'))
lci_lib_rms.statistics()

In [ ]:
if len(list(lci_lib_rms.unlinked)) == 0:
    lci_lib_rms.write_database()

# LCI from premise

## Lithium

In [ ]:
# LCI from Schenker et al (2022) 
lithium = LCI_DIR / 'from_premise' / 'lci-lithium.xlsx'
lithium = bi.ExcelImporter(lithium)
# Apply the necessary strategies
lithium.apply_strategies()

In [ ]:
# we match based on the name, reference product and location
lithium.statistics()

In [ ]:
# We have some unlinked exchanges, let's see which ones
for u in list(lithium.unlinked):
    print(u["name"], u.get("location"), u.get("categories"))

In [ ]:
# Let's try to link them with EI and biosphere
lithium.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
lithium.match_database("biosphere3", fields=('name', 'unit', 'categories'))
lithium.statistics()

In [ ]:
[u for u in lithium.unlinked if u["type"] == "technosphere"]

In [ ]:
[u for u in lithium.unlinked if u["type"] == "biosphere"]

3 ways to deal with this 
1. manually fix this (i.e., modify the exchange name in the Excel file),
2. go over imp.data(list), iterate through the exchanges and find Argon-40 and replace it with Argon
3. create a migration file for translating ecoinvent 3.9 flows to 3.10

### Migration from ei 3.8 to 3.10

The data is from ecoinvent 3.8 and we have 3.10. We create a mapping dictionary, and use it to create a `Migration` object.

In [ ]:
migration_38 = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("market for neutralising agent, sodium hydroxide-equivalent", 
             "neutralising agent, sodium hydroxide-equivalent", 
             "GLO"),
            {"location": "RER"}
        ),

        (
            ("market for soda ash, light, crystalline, heptahydrate", 
             "soda ash, light, crystalline, heptahydrate", 
             "GLO", ""),
            {"name": "market for soda ash, light", 
             "reference product": "soda ash, light", 
             "location": "RER"}
        ),

        (
            ("market for sodium hydroxide, without water, in 50% solution state", 
             "sodium hydroxide, without water, in 50% solution state", 
             "GLO"),
            {"location": "RER"}
        ),

         (
            ("electricity, high voltage, production mix", 
             "electricity, high voltage", 
             "CN-QH"),
            {"location": "CN-CCG"}
        ), 

        (
            ("Lithium, in ground", "", "", ("natural resource', 'in ground",)),
            {
                "name": "Lithium",
            },
        ), 

        (
            ("Sodium", "", "", ("water",)),
            {
                "name": "Sodium I",
            },
        ), 


        (
            ("Particulates, > 2.5 um, and < 10um", "", "", ("air",)),
            {
                "name": "Particulate Matter, > 2.5 um, and < 10um",
            },
        ), 

        (
            ("Particulates, < 2.5 um", "", "", ("air",)),
            {
                "name": "Particulate Matter, < 2.5 um",
            },
        )
        
    ],
}

In [ ]:
bi.Migration(name="ei3.8-3.10").write(data=migration_38, description="ei 3.8 to 3.10")

In [ ]:
"ei3.8-3.10" in bi.migrations

In [ ]:
bi.Migration("ei3.8-3.10")

We apply the migration on our imported data.

In [ ]:
lithium.data = bi.strategies.migrate_exchanges(
    db=lithium.data,
    migration="ei3.8-3.10"
)

In [ ]:
lithium.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
lithium.match_database("biosphere3", fields=('name', 'unit', 'categories'))
lithium.statistics()

In [ ]:
if len(list(lithium.unlinked)) == 0:
    lithium.write_database()

In [ ]:
bd.databases

## Cobalt

In [ ]:
# LCI from ??
cobalt = LCI_DIR / 'from_premise' / 'lci-cobalt.xlsx'
cobalt = bi.ExcelImporter(cobalt)
# Apply the necessary strategies
cobalt.apply_strategies()

In [ ]:
# we match based on the name, reference product and location
cobalt.match_database(fields=('name', 'reference product', 'unit', 'location')) 
cobalt.statistics()

In [ ]:
# Let's try to link them with EI
cobalt.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
cobalt.match_database("biosphere3", fields=('name', 'unit', 'categories'))
cobalt.statistics()

In [ ]:
[u for u in cobalt.unlinked if u["type"] == "technosphere"]

In [ ]:
[u for u in cobalt.unlinked if u["type"] == "biosphere"]

### Migration from ei ? to 3.10

In [ ]:
migration_3 = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("market for sodium hydroxide, without water, in 50% solution state", 
             "sodium hydroxide, without water, in 50% solution state", 
             "GLO"),
            {"location": "RER"}
        ),

        (
            ("Sodium", "", "", ("water",)),
            {
                "name": "Sodium I",
            },
        ), 

        (
            ("Cobalt, in ground", "", "", ("'natural resource', 'in ground'",)),
            {
                "name": "Cobalt",
            },
        ), 

        (
            ("Copper, in ground", "", "", ("'natural resource', 'in ground'",)),
            {
                "name": "Copper",
            },
        ), 

        (
            ("Particulates, > 2.5 um, and < 10um", "", "", ("air",)),
            {
                "name": "Particulate Matter, > 2.5 um, and < 10um",
            },
        ), 

        (
            ("Particulates, < 2.5 um", "", "", ("air",)),
            {
                "name": "Particulate Matter, < 2.5 um",
            },
        )
        
    ],
}

In [ ]:
bi.Migration(name="ei3.?-3.10").write(data=migration_3, description="ei 3.? to 3.10")

In [ ]:
cobalt.data = bi.strategies.migrate_exchanges(
    db=cobalt.data,
    migration="ei3.?-3.10"
)

In [ ]:
cobalt.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
cobalt.match_database("biosphere3", fields=('name', 'unit', 'categories'))
cobalt.statistics()

In [ ]:
if len(list(cobalt.unlinked)) == 0:
    cobalt.write_database()

## Graphite 

In [ ]:
# LCI from multiple sources
graphite = LCI_DIR / 'from_premise' / 'lci-graphite.xlsx'
graphite = bi.ExcelImporter(graphite)
graphite.apply_strategies()

In [ ]:
graphite.match_database(fields=('name', 'reference product', 'unit', 'location')) 
graphite.statistics()

In [ ]:
# Let's try to link them with EI
graphite.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
graphite.match_database("biosphere3", fields=('name', 'unit', 'categories'))
graphite.statistics()

In [ ]:
[u for u in graphite.unlinked if u["type"] == "technosphere"]

In [ ]:
[u for u in graphite.unlinked if u["type"] == "biosphere"]

## Migration from ei 3.? to 3.10

In [ ]:
migration_graphite = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("coking", 
             "coal gas", 
             "ROW"),
            {"name": "coke production"}
        ),
        (
            ("Particulates, > 10 um", "", "", ("air",)),
            {
                "name": "Particulate Matter, > 10 um",
            },
        ),

        (
            ("Oil, crude, in ground", "", "", ("'natural resource', 'in ground'",)),
            {
                "name": "Oil, crude",
            },
        ), 
        
    ],
}

In [ ]:
bi.Migration(name="graphite").write(data=migration_graphite, description="graphite")

In [ ]:
graphite.data = bi.strategies.migrate_exchanges(
    db=graphite.data,
    migration="graphite"
)

In [ ]:
graphite.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
graphite.match_database("biosphere3", fields=('name', 'unit', 'categories'))
graphite.statistics()

In [ ]:
if len(list(graphite.unlinked)) == 0:
    graphite.write_database()

# First LCA and contribution analysis 

In [ ]:
db = bd.Database("lithium")

In [ ]:
[a["name"] for a in db]

In [ ]:
activity = db.search('lithium carbonate production, from Salar de Olaroz')[0]
activity

In [ ]:
method = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')

In [ ]:
lca = bc.LCA({activity:1}, method)
lca.lci()
lca.lcia()
lca.score

In [ ]:
rev_prod, rev_act, rev_bio = lca.reverse_dict()

In [ ]:
results_by_activity = (lca.characterized_inventory.sum(axis=0)).A1

In [ ]:
# Create a list of names in columns
list_of_names_in_columns = [
    bd.get_activity(rev_prod[col])['name'] 
    for col in range((lca.characterized_inventory.sum(axis=0)).shape[1])
]

In [ ]:
pd.Series(index=list_of_names_in_columns, data=results_by_activity).sort_values(ascending=False).head(10)

In [ ]:
# Same analysis but streamlined with bw2analyzer
import bw2analyzer as ba

In [ ]:
pd.DataFrame(
    [(x, y, z["name"]) for x, y, z in ba.ContributionAnalysis().annotated_top_processes(lca=lca)],
    columns=["score", "quantity", "name"]
)

In [ ]:
pd.DataFrame(
    [(x, y, z["name"]) for x, y, z in ba.ContributionAnalysis().annotated_top_emissions(lca=lca)],
    columns=["score", "quantity", "name"]
)